In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
import time
import os
import csv
import torch
from validate import validate
from networks.resnet import resnet50
from options.test_options import TestOptions
import networks.resnet as resnet
import numpy as np
import random
import pandas as pd

DetectionTests = {
                'ForenSynths': { 'dataroot'   : '../grads/ForenSynths/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

           'GANGen-Detection': { 'dataroot'   : '../grads/GANGen-Detection/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

         'DiffusionForensics': { 'dataroot'   : '../grads/DiffusionForensics/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

        'UniversalFakeDetect': { 'dataroot'   : '../grads/UniversalFakeDetect/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

        'Diffusion1kStep': { 'dataroot'   : '../grads/Diffusion1kStep/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

                 }


opt = TestOptions()

# params
opt.model_path = './weights/LGrad-1class-Trainon-Progan_horse.pth'
opt.batch_size = 1024
opt.isTrain = False
opt.mode = 'binary'
opt.class_bal = False
opt.eval = True
opt.num_threads = 8
opt.rz_interp = ['bilinear']
opt.loadSize = 256
opt.blur_prob = 0
opt.jpg_prob = 0
opt.cropSize = 224

print(f'Model_path {opt.model_path}')
torch.cuda.set_device('cuda:0')

# get model
model = resnet50(num_classes=1)
model.load_state_dict(torch.load(opt.model_path, map_location='cuda:0'))
model.cuda()
model.eval()

results = []

for testSet in DetectionTests.keys():
    dataroot = DetectionTests[testSet]['dataroot']
    print(testSet)

    accs = []
    aps = []
    print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
    for v_id, val in enumerate(os.listdir(dataroot)):
        opt.dataroot = '{}/{}'.format(dataroot, val)
        opt.classes  = '' #os.listdir(opt.dataroot) if multiclass[v_id] else ['']
        opt.no_resize = DetectionTests[testSet]['no_resize']
        opt.no_crop   = DetectionTests[testSet]['no_crop']
        acc, ap, _, _, _, _ = validate(model, opt)
        accs.append(acc)
        aps.append(ap)
        print("({} {:12}) acc: {:.1f}; ap: {:.1f}".format(v_id, val, acc*100, ap*100))
        results.append({'group': testSet, 'id': v_id, 'dataset': val, 'accuracy': acc * 100, 'average_precision': ap * 100})
    results.append({'group': testSet, 'id': v_id + 1, 'dataset': 'Mean', 'accuracy': np.array(accs).mean() * 100, 'average_precision': np.array(aps).mean() * 100})    #TODO: изменить dataframe
    print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id+1,'Mean', np.array(accs).mean()*100, np.array(aps).mean()*100))
    print('*'*25) 

df = pd.DataFrame(results)
df.to_csv('LGrad_CNNDetection_results.csv', index=False)


Model_path ./weights/LGrad-1class-Trainon-Progan_horse.pth


/tmp/ipykernel_341248/1610269258.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(opt.model_path, map_location='cuda:0'))


ForenSynths
2024_07_31_14_52_27
(0 progan      ) acc: 99.1; ap: 99.9
(1 stylegan    ) acc: 94.6; ap: 99.2
(2 cyclegan    ) acc: 83.1; ap: 92.9
(3 seeingdark  ) acc: 52.8; ap: 43.8
(4 stargan     ) acc: 99.2; ap: 99.9
(5 imle        ) acc: 52.2; ap: 82.4
(6 crn         ) acc: 52.2; ap: 82.1
(7 whichfaceisreal) acc: 62.8; ap: 67.0
(8 san         ) acc: 49.2; ap: 48.7
(9 stylegan2   ) acc: 92.5; ap: 99.1
(10 deepfake    ) acc: 62.7; ap: 76.6
(11 gaugan      ) acc: 69.5; ap: 80.3
(12 biggan      ) acc: 78.1; ap: 86.5
(13 Mean      ) acc: 72.9; ap: 81.4
*************************
GANGen-Detection
2024_07_31_14_57_57
(0 AttGAN      ) acc: 64.7; ap: 86.2
(1 BEGAN       ) acc: 59.7; ap: 81.8
(2 MMDGAN      ) acc: 54.4; ap: 77.3
(3 STGAN       ) acc: 53.8; ap: 87.0
(4 SNGAN       ) acc: 66.1; ap: 91.5
(5 InfoMaxGAN  ) acc: 66.4; ap: 90.8
(6 CramerGAN   ) acc: 56.4; ap: 80.1
(7 RelGAN      ) acc: 83.2; ap: 96.3
(8 S3GAN       ) acc: 74.9; ap: 81.3
(9 Mean      ) acc: 64.4; ap: 85.8
**************

In [10]:
df

,group,id,dataset,accuracy,average_precision
0,ForenSynths,0,progan,99.062500,99.859758
1,ForenSynths,1,stylegan,94.556452,99.228835
2,ForenSynths,2,cyclegan,83.085809,92.850508
3,ForenSynths,3,seeingdark,52.777778,43.774994
4,ForenSynths,4,stargan,99.168347,99.922530
5,ForenSynths,5,imle,52.209596,82.382373
6,ForenSynths,6,crn,52.201705,82.110545
7,ForenSynths,7,whichfaceisreal,62.850000,67.009550
8,ForenSynths,8,san,49.218750,48.721341
9,ForenSynths,9,stylegan2,92.452117,99.068017
